In [8]:
%load_ext autoreload
%autoreload 2
import mindaffectBCI.decoder
from mindaffectBCI.decoder.UtopiaDataInterface import butterfilt_and_downsample
from mindaffectBCI.decoder.devent2stimsequence import devent2stimSequence, upsample_stimseq
from mindaffectBCI.decoder.decodingSupervised import decodingSupervised
from mindaffectBCI.decoder.decodingCurveSupervised import decodingCurveSupervised, plot_decoding_curve
from mindaffectBCI.decoder.model_fitting import BaseSequence2Sequence, MultiCCA
from mindaffectBCI.decoder.scoreOutput import dedupY0
from mindaffectBCI.decoder.updateSummaryStatistics import updateSummaryStatistics, plot_summary_statistics, plot_erp
import os
import time
import numpy as np
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def strip_unused_stim_chan(Y):
    """
    Strip unused stimulation channels, i.e. those that register no non-zero values at any time in any trial.
    NB this approach assumes that channel 0 ("objID" 0) is special, and is always kept
    """
    used_y = np.any(Y.reshape((-1, Y.shape[-1])), 0)
    used_y[0] = True # ensure objID=0 is always used..  NB - might not be appropriate for us
    Y = Y[..., used_y]
    return Y, used_y

In [3]:
def dataset_to_XY_ndarrays(dataset):
    """
    Convert varitable trial length dataset with stimulus information to fixed wize sklearn-compatible (X,Y) ndarrays.
    dataset: num_trials-lenth list of 2-lists of (len, dim) arrays. 1st element is the electrode signals, 2nd is the stimulation signals.
            Last channel (column) of electrode signal arrays holds the timestamp per sample.
            Last channel (column) of the stimulation arrays holds the timestamp per stimulus event
    """
    # get length of each trial
    tr_lens = [tl[0].shape[0] for tl in dataset]
    tr_stim_lens = [tl[1].shape[0] for tl in datset]
    
    # set array trial length to 90th percentile
    tr_len = int(np.percentile(tr_lens, 90))
    tr_stim_len = max(20, int(np.percentile(tr_stim_lens, 90)))
    # filter the trials to only be the ones long enough to be worth processing
    dataset = [d for d in dataset if d[0].shape[0] > tr_len//2 and d[1].shape[0] > tr_stim_len//2]
    
    # map to single fixed size matrix + upsample stimulus to the EEG/EROS sample rate
    Y = np.zeros((len(dataset), tr_len, 256), dtype=dataset[0][1].dtype)
    X = np.zeros((len(dataset), tr_len, dataset[0][0].shape[-1]-1), dtype=dataset[0][0].dtype)
    X_ts = np.zeros((len(dataset), tr_len), dtype=int)
    Y_ts = np.zeros((len(dataset), tr_len), dtype=int)
    for ti, (data, stimulus) in enumerate(dataset):
        # extract the data, remove the timestamp channel and insert into the ndarray
        # guard for slightly different sizes
        if X.shape[1] <= data.shape[0]:
            X[ti, :, :] = data[:X.shape[1], :-1]
            X_ts[ti, :] = data[:X.shape[1], -1]
        else: # pad end with final value
            X[ti, :data.shape[0], :] = data[:, :-1]
            X[ti, data.shape[0]:, :] = data[-1, :-1]
            X_ts[ti, :data.shape[0]] = data[:, -1]
            
        # STIMULUS UPSAMPLING: needs a closer look
        data_ts = data[:, -1] # data timestamp per sample
        stimulus_ts = stimulus[:, -1] # stimulus timestamp per stimulus event
        stimulus, data_i = upsample_stimseq(data_ts, stimulus[:, :-1], stimulus_ts)
        # store -- compensating for any variable trial lengths
        if Y.shape[1] < stimulus.shape[0]:  # long trial
            Y[ti, :, :] = stimulus[:Y.shape[1], :]
        else:  # short trial
            Y[ti, :stimulus.shape[0], :] = stimulus
        # record stim_ts @ this data_ts
        tmp = data_i < Y.shape[1]
        Y_ts[ti, data_i[tmp]] = stimulus[tmp]
        
    return X, Y, X_ts, Y_ts
            
    

In [4]:
def calibrate(start, end, fs, clsfr: BaseSequence2Sequence, cv=2, previous_dataset=None):
    
#     dataset = getCalibration_dataset(start, end)
    if previous_dataset:
        dataset.extend(previous_dataset)
        
    if dataset:
        # convert msgs to ndarrays
        X, Y, X_ts, Y_ts = dataset_to_XY_ndarrays(dataset)
        
        try:
            pickle.dump(dict(X=X, Y=Y, X_ts=X_ts, Y_ts=Y_ts, fs=fs),
                       open('calibration_data.pk', 'wb'))
        except:
            print('Error saving calibration data')
        
    if X is None or Y is None:
        return None, None
    
    Y, used_idx = strip_unused_stim_chan(Y)
    
    
    # call the classifier fit method
    print(f"Training datset = {X.shape}, {Y.shape}")
    cvscores = clsfr.cv_fit(X, Y, cv=cv)
    score = np.mean(cvscores['test_score'])
    print(f"classifier={clsfr} => {score}")
    decoding_curve = decodingCurveSupervised(cvscores['estimator'], nInt=(10, 10),
                                            priorsigma=(clsfr.sigma0_, clsfr.priorweight),
                                            softmaxscale=clsfr.softmaxscale_)
    # extract the final estimated performance
    print(f"decoding curve {decoding_curve[1]}")
    print(f"score {score}")
    perr = decoding_curve[1][-1] if len(decoding_curve) > 1 else 1-score
    if CALIBRATIONPLOTS:
        try:
        #if True:
            import matplotlib.pyplot as plt
            plt.figure(1)
            clsfr.plot_model(fs=fs)
            plt.suptitle('Factored Model')
            plt.figure(2)
            plot_decoding_curve(*decoding_curve)
            plt.suptitle('Decoding Curve')
            #  from analyse_datasets import debug_test_dataset
            #  debug_test_dataset(X,Y,None,fs=ui.fs)
            plt.figure(3) # plot the CCA info
            Y_true = clsfr.stim2event(Y)
            Y_true = Y_true[...,0:1,:]
            Cxx, Cxy, Cyy = updateSummaryStatistics(X,Y_true,tau=clsfr.tau)
            plot_summary_statistics(Cxx,Cxy,Cyy,clsfr.evtlabs,fs=fs)
            plt.suptitle("Summary Statistics")
            try:
                pickle.dump(dict(Cxx=Cxx, Cxy=Cxy, Cyy=Cyy, evtlabs=clsfr.evtlabs, fs=fs),
                            open('summary_statistics.pk','wb'))
            except:
                print('Error saving cal data')
            plt.figure(4)
            plot_erp(Cxy,evtlabs=clsfr.evtlabs,fs=fs)
            plt.suptitle("Event Related Potential (ERP)")
            plt.show(block=False)
            # save figures
            logsdir = os.path.join(os.path.dirname(os.path.abspath(__file__)),'../../logs/')
            plt.figure(1)
            plt.savefig(os.path.join(logsdir,'model_{}.png'.format(uname)))
            plt.figure(2)
            plt.savefig(os.path.join(logsdir,'decoding_curve_{}.png'.format(uname)))
            plt.figure(3)
            plt.savefig(os.path.join(logsdir,'summary_statistics_{}.png'.format(uname)))
            plt.figure(4)
            plt.savefig(os.path.join(logsdir,'erp_{}.png'.format(uname)))
        except:
            pass
        
    return dataset, X, Y

In [5]:
def predict(X, Y, X_ts, Y_ts, clsfr: BaseSequence2Sequence, prev_stimulus=None, special=False):
    """
    Given the current trials data, apply the classifier and decoder to make target predictions
    """
    Y, used_idx = strip_unused_stim_chan(Y)
    if special:
        # if channel 0 of Y corresponds to the special "obj0", then it may be a copy of another, so remove it
        Y = dedupY0(Y, zerodup=False, yfeatdim=False)
    
    # NB don't need to upsample as in original repo since we are offline and this has occurred already
    Fy_1 = clsfr.predict(X, Y, prevY=prev_stimulus)  # CHECK THIS prevY arg
    # map back to 256
    Fy = np.zeros(Fy_1.shape[:-1] + (256,), dtype=Fy_1.dtype)
    Fy[..., used_idx] = Fy_1
    
    return Fy
    

Perform calibration and prediction on an incrementing train-test split fraction. Initially there is only one trial in the train split and it might correspond to a special calibration stimulation sequence

In [ ]:
num_trials = 10
trial_len = 200 * 5  # just assume exact same length for all trials
nch = 4
data_list = []
ts_list = []
for tr in range(num_trials):
    trial = np.zeros((trial_len, nch))
    ts = np.zeros((trial_len, ))  # also assume ts lens are same as data lens
    for sa in range(trial_len):
        trial[sa, :] = np.random.standard_normal((nch,))
        
    data_list.append(trial)

In [10]:
from mindaffectBCI.noisetag import Noisetag
# objIDs=list(range(1,10))
nt = Noisetag()
nt.connect()
nt.setnumActiveObjIDs(10)
nt.startExpt(framesperbit=4)
nframe=0
isi=1/60
while True:
    try :
        # update the stimulus state w.r.t. current time
        nt.updateStimulusState(nframe)
        # get the stimulus state info we whould display
        ss,ts,objIDs,sendEvent=nt.getStimulusState()
        print(ss, ts, objIDs)
        print('a')
        # update the display drawing
#         doFrame(nframe,ss,ts,objIDs)            
        # send info about what we did to the decoder
        nt.sendStimulusState()
        # simulate waiting for the flip
        time.sleep(isi)
    except StopIteration :
        # this event is raised when the stimulus sequence is finished
        break
    nframe=nframe+1
        

waitFor: 120
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a

1.wait
flicker: 60 frames, tgt -1
highlight: tgtidx=-1 nframes=60
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8

[1, 0, 0, 1, 0, 1, 0, 0, 0, 0] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 0, 0, 1, 0, 1, 0, 0, 0, 0] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 0, 0, 1, 0, 1, 0, 0, 0, 0] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 0, 0, 1, 0, 1, 0, 0, 0, 0] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 1, 1, 0, 1, 0, 1, 1, 1] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 1, 1, 0, 1, 0, 1, 1, 1] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 1, 1, 0, 1, 0, 1, 1, 1] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 1, 1, 0, 1, 0, 1, 1, 1] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 0, 0, 1, 1, 0, 1, 1, 1, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 0, 0, 1, 1, 0, 1, 1, 1, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 0, 0, 1, 1, 0, 1, 1, 1, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 1

[0, 1, 0, 0, 1, 0, 1, 1, 0, 1] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 1, 0, 0, 1, 0, 1, 1, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 1, 0, 0, 1, 0, 1, 1, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 1, 0, 0, 1, 0, 1, 1, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 0, 1, 0, 0, 1, 0, 1, 1, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 1, 0, 1, 1, 0, 1, 0, 0, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 1, 0, 1, 1, 0, 1, 0, 0, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 1, 0, 1, 1, 0, 1, 0, 0, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 1, 0, 1, 1, 0, 1, 0, 0, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 0, 0, 1, 0, 0, 1, 0, 1, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 0, 0, 1, 0, 0, 1, 0, 1, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 0, 0, 1, 0, 0, 1, 0, 1, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[1, 0, 0, 1, 0, 0, 1, 0, 1, 1] 1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 1, 1, 0, 1, 1, 0, 1, 0, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
[0, 1, 1, 0, 1, 1, 0, 1, 0, 0] 0 [1, 2, 3, 4, 5, 6, 7, 8, 9, 1

None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9

...

Socket error[Errno 35] Resource temporarily unavailable#35
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a

3.feedback
Socket error[Errno 35] Resource temporarily unavailable#35
11459ms pred:-1 sel:0  x
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a

Start Pred: 18/20
tgtidx=-1
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
None -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
2.stim, tgt:-1
flicker: 600 frames, tgt -1
Socket error[Errno 35] Resource temporarily unavailable#35
 with selection
[0, 1, 1, 1, 1, 1, 1, 0, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[0, 1, 1, 1, 1, 1, 1, 0, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[0, 1, 1, 1, 1, 1, 1, 0, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable

Socket error[Errno 35] Resource temporarily unavailable#35
[1, 0, 0, 1, 0, 1, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 0, 0, 1, 0, 1, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 0, 0, 1, 0, 1, 0, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 1, 0, 0, 1, 0, 1, 0, 0, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[0, 0, 1, 1, 0, 1, 0, 1, 1, 1] -1 [1, 2, 3, 4, 5, 6,

Socket error[Errno 35] Resource temporarily unavailable#35
[0, 1, 1, 1, 0, 1, 1, 1, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[0, 1, 0, 0, 0, 1, 0, 0, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[0, 1, 0, 0, 0, 1, 0, 0, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[0, 1, 0, 0, 0, 1, 0, 0, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[0, 1, 0, 0, 0, 1, 0, 0, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 0, 1, 1, 1, 0, 1, 1, 0, 1] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 0, 1, 1, 1, 0, 1, 1, 0, 1] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 0, 1, 1, 1, 0, 1, 1, 0, 1] -1 [1, 2, 3, 4, 5, 6,

Socket error[Errno 35] Resource temporarily unavailable#35
[0, 0, 1, 0, 0, 1, 0, 1, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[0, 0, 1, 0, 0, 1, 0, 1, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[0, 0, 1, 0, 0, 1, 0, 1, 1, 0] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 1, 0, 1, 1, 0, 1, 0, 0, 1] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 1, 0, 1, 1, 0, 1, 0, 0, 1] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 1, 0, 1, 1, 0, 1, 0, 0, 1] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 1, 0, 1, 1, 0, 1, 0, 0, 1] -1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
a
Socket error[Errno 35] Resource temporarily unavailable#35
[1, 0, 0, 1, 0, 0, 1, 0, 1, 1] -1 [1, 2, 3, 4, 5, 6,

KeyboardInterrupt: 

In [38]:
len(nt.noisecode.stimSeq[68])

47

In [39]:
nt.startCalibration()

ValueError: 

In [9]:
a = np.array([[[i + 2*j + 8*k for i in range(3)] for j in range(3)] for k in range(3)])
a

array([[[ 0,  1,  2],
        [ 2,  3,  4],
        [ 4,  5,  6]],

       [[ 8,  9, 10],
        [10, 11, 12],
        [12, 13, 14]],

       [[16, 17, 18],
        [18, 19, 20],
        [20, 21, 22]]])

In [15]:
oi = a[..., 0, :]
print(oi)
print(oi.shape)
oi = a[..., 0:1, :]
oi.shape

[[ 0  1  2]
 [ 8  9 10]
 [16 17 18]]
(3, 3)


(3, 1, 3)

In [30]:
oi = np.any(a, 0)

In [31]:
oi

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [33]:
used = np.any(a.reshape((-1, a.shape[-1])), 0)

In [34]:
used

array([ True,  True,  True])

In [35]:
res = a.reshape((-1, a.shape[-1]))

In [36]:
res

array([[ 0,  1,  2],
       [ 2,  3,  4],
       [ 4,  5,  6],
       [ 8,  9, 10],
       [10, 11, 12],
       [12, 13, 14],
       [16, 17, 18],
       [18, 19, 20],
       [20, 21, 22]])

In [37]:
res.shape

(9, 3)